In [3]:
pip install pandas scikit-learn  imbalanced-learn  xgboost pandas scikit-learn


  Using cached pandas-2.2.3-cp310-cp310-win_amd64.whl.metadata (19 kB)
  Using cached scikit_learn-1.6.1-cp310-cp310-win_amd64.whl.metadata (15 kB)
  Using cached imbalanced_learn-0.13.0-py3-none-any.whl.metadata (8.8 kB)
  Using cached xgboost-2.1.4-py3-none-win_amd64.whl.metadata (2.1 kB)
  Using cached pytz-2025.1-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2025.1-py2.py3-none-any.whl.metadata (1.4 kB)
  Using cached scipy-1.15.2-cp310-cp310-win_amd64.whl.metadata (60 kB)
  Using cached joblib-1.4.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached sklearn_compat-0.1.3-py3-none-any.whl.metadata (18 kB)
Using cached pandas-2.2.3-cp310-cp310-win_amd64.whl (11.6 MB)
Using cached scikit_learn-1.6.1-cp310-cp310-win_amd64.whl (11.1 MB)
Using cached imbalanced_learn-0.13.0-py3-none-any.whl (238 kB)
Using cached xgboost-2.1.4-py3-none-win_amd64.whl (124.9 MB)
Using cached joblib-1.4.2-py3-none-any.whl (301 kB)
Using cached pytz-2025.1-py2.py3-none-any.whl (507 kB)
Using cach

In [2]:
pip install torch==2.0.1 torchvision==0.15.2 torchaudio==2.0.2


  Using cached torch-2.0.1-cp310-cp310-win_amd64.whl.metadata (23 kB)
  Using cached torchvision-0.15.2-cp310-cp310-win_amd64.whl.metadata (11 kB)
  Using cached torchaudio-2.0.2-cp310-cp310-win_amd64.whl.metadata (1.2 kB)
  Using cached filelock-3.17.0-py3-none-any.whl.metadata (2.9 kB)
  Using cached sympy-1.13.3-py3-none-any.whl.metadata (12 kB)
  Using cached networkx-3.4.2-py3-none-any.whl.metadata (6.3 kB)
  Using cached jinja2-3.1.6-py3-none-any.whl.metadata (2.9 kB)
  Using cached numpy-2.2.3-cp310-cp310-win_amd64.whl.metadata (60 kB)
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached pillow-11.1.0-cp310-cp310-win_amd64.whl.metadata (9.3 kB)
  Using cached MarkupSafe-3.0.2-cp310-cp310-win_amd64.whl.metadata (4.1 kB)
  Using cached charset_normalizer-3.4.1-cp310-cp310-win_amd64.whl.metadata (36 kB)
  Using cached idna-3.10-py3-none-any.whl.metadata (10 kB)
  Using cached urllib3-2.3.0-py3-none-any.whl.metadata (6.5 kB)
  Using cached certifi-2025.1.

In [4]:
import flwr as fl
import xgboost as xgb
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [5]:
# Load the dataset
df = pd.read_csv("farmighamdataset.csv")

In [ ]:
# Define target column
target_column = 'TenYearCHD'

# Separate majority and minority classes
df_majority = df[df[target_column] == 0]
df_minority = df[df[target_column] == 1]

# Upsample the minority class
df_minority_upsampled = resample(df_minority,replace=True,    # Sample with replacement
n_samples=len(df_majority),random_state=42)  


# Combine majority and upsampled minority class
df_balanced = pd.concat([df_majority, df_minority_upsampled])

# Shuffle the dataset
df_balanced = df_balanced.sample(frac=1, random_state=42).reset_index(drop=True)


In [ ]:
#Print new class distribution
print("Balanced Class Distribution:\n", df_balanced[target_column].value_counts())

In [ ]:
# Store accuracies
client_accuracies = []

# Train a Random Forest model on each client
for client_id, client_df in enumerate(client_splits, start=1):  # Ensure client_id is 1,2,3
    # Split client data into features and labels
    X_client = client_df.drop(columns=[target_column])
    y_client = client_df[target_column]

    # Split into train and test sets
    X_train, X_test, y_train, y_test = train_test_split(X_client, y_client, test_size=0.2, random_state=42)

    # Ensure there is training data
    print(f"\nClient {client_id}: Train size = {len(X_train)}, Test size = {len(X_test)}")

    if len(X_train) == 0 or len(X_test) == 0:
        print(f"Client {client_id} has insufficient data for training. Skipping...")
        continue

    # Train a Random Forest model
    model = RandomForestClassifier(n_estimators=100, random_state=42)
    model.fit(X_train, y_train)

    # Evaluate the model
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)

    # Store accuracy
    client_accuracies.append((client_id, accuracy))
    print(f"Client {client_id}: Accuracy = {accuracy:.4f}")  # Print directly inside loop



In [ ]:
# Print all client accuracies at the end
print("\nFinal Client Accuracies:")
for client_id, acc in client_accuracies:
    print(f"Client {client_id}: Accuracy = {acc:.4f}")

In [3]:
import flwr as fl
import numpy as np

class SimpleClient(fl.client.NumPyClient):
    def get_parameters(self, config):
        """Generate and return random parameters."""
        params = np.random.rand(2, 2)  # Random 2x2 matrix
        print(f"📤 Sending parameters: {params}")
        return [params]  # Return as a list of NumPy arrays

    def fit(self, parameters, config):
        """Simulate training by returning the same parameters."""
        print(f"📥 Received parameters from server: {parameters}")
        return parameters, 1, {}  # Return the same parameters

    def evaluate(self, parameters, config):
        """Simulate evaluation by computing random metrics."""
        print("📊 Evaluating model")
        # Simulate evaluation by generating random metrics
        loss = float(np.random.rand())  # Random loss between 0 and 1
        accuracy = float(np.random.rand())  # Random accuracy between 0 and 1
        num_examples = 10  # Simulate evaluating on 10 examples
        metrics = {"accuracy": [accuracy], "loss": [loss]}  # Wrap metrics in a list
        return loss, num_examples, metrics

# Start Flower client
fl.client.start_numpy_client(
    server_address="127.0.0.1:8080",  # Server address
    client=SimpleClient(),
)

INFO flwr 2025-03-14 09:07:02,764 | grpc.py:50 | Opened insecure gRPC connection (no certificates were passed)
DEBUG flwr 2025-03-14 09:07:02,831 | connection.py:39 | ChannelConnectivity.IDLE
DEBUG flwr 2025-03-14 09:07:02,837 | connection.py:39 | ChannelConnectivity.READY


📤 Sending parameters: [[0.12200337 0.39240096]
 [0.18209185 0.40720201]]


DEBUG flwr 2025-03-14 09:07:32,811 | connection.py:39 | ChannelConnectivity.IDLE
DEBUG flwr 2025-03-14 09:07:33,022 | connection.py:113 | gRPC channel closed


_MultiThreadedRendezvous: <_MultiThreadedRendezvous of RPC that terminated with:
	status = StatusCode.UNAVAILABLE
	details = "IOCP/Socket: Connection reset (An existing connection was forcibly closed by the remote host.
 -- 10054)"
	debug_error_string = "UNKNOWN:Error received from peer ipv4:127.0.0.1:8080 {created_time:"2025-03-14T03:37:32.7660292+00:00", grpc_status:14, grpc_message:"IOCP/Socket: Connection reset (An existing connection was forcibly closed by the remote host.\r\n -- 10054)"}"
>

In [6]:
import flwr as fl
import numpy as np

class SimpleClient(fl.client.NumPyClient):
    def get_parameters(self, config):
        """Generate and return random parameters."""
        params = np.random.rand(2, 2)  # Random 2x2 matrix
        print(f"📤 Sending parameters: {params}")
        return [params]  # Return as a list of NumPy arrays

    def evaluate(self, parameters, config):
        """Simulate evaluation by returning fixed metrics."""
        print("📊 Evaluating model")
        # Return loss, number of examples, and metrics
        loss = 0.9  # Fixed loss value
        num_examples = 1  # Number of examples used for evaluation
        metrics = {"accuracy": [0.9]}  # Wrap metrics in a list
        return float(loss), num_examples, metrics

# Start Flower client
fl.client.start_numpy_client(
    server_address="127.0.0.1:8080",  # Server address
    client=SimpleClient(),
)

INFO flwr 2025-03-14 09:10:30,544 | grpc.py:50 | Opened insecure gRPC connection (no certificates were passed)
DEBUG flwr 2025-03-14 09:10:30,592 | connection.py:39 | ChannelConnectivity.IDLE
DEBUG flwr 2025-03-14 09:10:30,600 | connection.py:39 | ChannelConnectivity.READY


📤 Sending parameters: [[0.30361417 0.71865858]
 [0.71852555 0.84826934]]


DEBUG flwr 2025-03-14 09:10:53,378 | connection.py:39 | ChannelConnectivity.IDLE
DEBUG flwr 2025-03-14 09:10:53,600 | connection.py:113 | gRPC channel closed


_MultiThreadedRendezvous: <_MultiThreadedRendezvous of RPC that terminated with:
	status = StatusCode.UNAVAILABLE
	details = "IOCP/Socket: Connection reset (An existing connection was forcibly closed by the remote host.
 -- 10054)"
	debug_error_string = "UNKNOWN:Error received from peer ipv4:127.0.0.1:8080 {grpc_message:"IOCP/Socket: Connection reset (An existing connection was forcibly closed by the remote host.\r\n -- 10054)", grpc_status:14, created_time:"2025-03-14T03:40:53.3281689+00:00"}"
>

In [7]:
import flwr as fl
import numpy as np

class SimpleClient(fl.client.NumPyClient):
    def get_parameters(self, config):
        params = np.random.rand(2, 2)
        print(f"📤 Sending parameters: {params}")
        return [params]

    def evaluate(self, parameters, config):
        print("📊 Evaluating model")
        return 0.9, 1, {"accuracy": [0.9]}  # Wrap metrics in a list

fl.client.start_numpy_client(
    server_address="127.0.0.1:8080",
    client=SimpleClient(),
)

INFO flwr 2025-03-14 09:11:03,008 | grpc.py:50 | Opened insecure gRPC connection (no certificates were passed)
DEBUG flwr 2025-03-14 09:11:03,050 | connection.py:39 | ChannelConnectivity.IDLE
DEBUG flwr 2025-03-14 09:11:03,063 | connection.py:39 | ChannelConnectivity.READY


📤 Sending parameters: [[0.18012066 0.43339929]
 [0.310707   0.71824855]]


DEBUG flwr 2025-03-14 09:26:39,284 | connection.py:113 | gRPC channel closed


KeyboardInterrupt: 

In [1]:
import flwr as fl
import numpy as np

# Define a simple client
class SimpleClient(fl.client.NumPyClient):
    def get_parameters(self, config):
        return [np.array([0.1, 0.2, 0.3])]  # Dummy model parameters

    def fit(self, parameters, config):
        print("Training model...")
        return self.get_parameters(config), len(parameters), {}

    def evaluate(self, parameters, config):
        print("Evaluating model...")
        return 0.5, len(parameters), {}  # Dummy loss

# Start the client
if __name__ == "__main__":
    fl.client.start_numpy_client(server_address="127.0.0.1:8080", client=SimpleClient())


	Instead, use `flwr.client.start_client()` by ensuring you first call the `.to_client()` method as shown below: 
	flwr.client.start_client(
		server_address='<IP>:<PORT>',
		client=FlowerClient().to_client(), # <-- where FlowerClient is of type flwr.client.NumPyClient object
	)
	Using `start_numpy_client()` is deprecated.

            This is a deprecated feature. It will be removed
            entirely in future versions of Flower.
        
	Instead, use the `flower-supernode` CLI command to start a SuperNode as shown below:

		$ flower-supernode --insecure --superlink='<IP>:<PORT>'

	To view all available options, run:

		$ flower-supernode --help

	Using `start_client()` is deprecated.

            This is a deprecated feature. It will be removed
            entirely in future versions of Flower.
        
INFO :      
INFO :      Received: get_parameters message 0b8fd41b-0b95-47fd-8954-23d21da0a0fa
INFO :      Sent reply


_MultiThreadedRendezvous: <_MultiThreadedRendezvous of RPC that terminated with:
	status = StatusCode.UNAVAILABLE
	details = "IOCP/Socket: Connection reset (An existing connection was forcibly closed by the remote host.
 -- 10054)"
	debug_error_string = "UNKNOWN:Error received from peer ipv4:127.0.0.1:8080 {created_time:"2025-03-14T04:42:45.8027609+00:00", grpc_status:14, grpc_message:"IOCP/Socket: Connection reset (An existing connection was forcibly closed by the remote host.\r\n -- 10054)"}"
>

In [ ]:
import flwr as fl
import numpy as np

class SimpleClient(fl.client.NumPyClient):
    def get_parameters(self, config):
        return [np.array([0.1, 0.2, 0.3])]

    def fit(self, parameters, config):
        return [p * 1.1 for p in parameters], len(parameters), {}

    def evaluate(self, parameters, config):
        return 0.5, len(parameters), {}

# Keep the client alive
while True:
    try:
        fl.client.start_client(
            server_address="127.0.0.1:8080",
            client=SimpleClient().to_client(),
        )
    except Exception as e:
        print(f"Client disconnected, retrying... {e}")


	Instead, use the `flower-supernode` CLI command to start a SuperNode as shown below:

		$ flower-supernode --insecure --superlink='<IP>:<PORT>'

	To view all available options, run:

		$ flower-supernode --help

	Using `start_client()` is deprecated.

            This is a deprecated feature. It will be removed
            entirely in future versions of Flower.
        
INFO :      
INFO :      Received: get_parameters message 6a09b91a-1967-4311-a276-8e91bbc34e54
INFO :      Sent reply
	Instead, use the `flower-supernode` CLI command to start a SuperNode as shown below:

		$ flower-supernode --insecure --superlink='<IP>:<PORT>'

	To view all available options, run:

		$ flower-supernode --help

	Using `start_client()` is deprecated.

            This is a deprecated feature. It will be removed
            entirely in future versions of Flower.
        


Client disconnected, retrying... <_MultiThreadedRendezvous of RPC that terminated with:
	status = StatusCode.UNAVAILABLE
	details = "IOCP/Socket: Connection reset (An existing connection was forcibly closed by the remote host.
 -- 10054)"
	debug_error_string = "UNKNOWN:Error received from peer ipv4:127.0.0.1:8080 {created_time:"2025-03-14T04:56:37.548125+00:00", grpc_status:14, grpc_message:"IOCP/Socket: Connection reset (An existing connection was forcibly closed by the remote host.\r\n -- 10054)"}"
>


	Instead, use the `flower-supernode` CLI command to start a SuperNode as shown below:

		$ flower-supernode --insecure --superlink='<IP>:<PORT>'

	To view all available options, run:

		$ flower-supernode --help

	Using `start_client()` is deprecated.

            This is a deprecated feature. It will be removed
            entirely in future versions of Flower.
        


Client disconnected, retrying... <_MultiThreadedRendezvous of RPC that terminated with:
	status = StatusCode.UNAVAILABLE
	details = "failed to connect to all addresses; last error: UNAVAILABLE: ipv4:127.0.0.1:8080: ConnectEx: Connection refused (No connection could be made because the target machine actively refused it.
 -- 10061)"
	debug_error_string = "UNKNOWN:Error received from peer  {created_time:"2025-03-14T04:56:39.9172378+00:00", grpc_status:14, grpc_message:"failed to connect to all addresses; last error: UNAVAILABLE: ipv4:127.0.0.1:8080: ConnectEx: Connection refused (No connection could be made because the target machine actively refused it.\r\n -- 10061)"}"
>


	Instead, use the `flower-supernode` CLI command to start a SuperNode as shown below:

		$ flower-supernode --insecure --superlink='<IP>:<PORT>'

	To view all available options, run:

		$ flower-supernode --help

	Using `start_client()` is deprecated.

            This is a deprecated feature. It will be removed
            entirely in future versions of Flower.
        


...

Client disconnected, retrying... <_MultiThreadedRendezvous of RPC that terminated with:
	status = StatusCode.UNAVAILABLE
	details = "failed to connect to all addresses; last error: UNAVAILABLE: ipv4:127.0.0.1:8080: ConnectEx: Connection refused (No connection could be made because the target machine actively refused it.
 -- 10061)"
	debug_error_string = "UNKNOWN:Error received from peer  {grpc_message:"failed to connect to all addresses; last error: UNAVAILABLE: ipv4:127.0.0.1:8080: ConnectEx: Connection refused (No connection could be made because the target machine actively refused it.\r\n -- 10061)", grpc_status:14, created_time:"2025-03-14T05:04:14.3371621+00:00"}"
>


	Instead, use the `flower-supernode` CLI command to start a SuperNode as shown below:

		$ flower-supernode --insecure --superlink='<IP>:<PORT>'

	To view all available options, run:

		$ flower-supernode --help

	Using `start_client()` is deprecated.

            This is a deprecated feature. It will be removed
            entirely in future versions of Flower.
        


Client disconnected, retrying... <_MultiThreadedRendezvous of RPC that terminated with:
	status = StatusCode.UNAVAILABLE
	details = "failed to connect to all addresses; last error: UNAVAILABLE: ipv4:127.0.0.1:8080: ConnectEx: Connection refused (No connection could be made because the target machine actively refused it.
 -- 10061)"
	debug_error_string = "UNKNOWN:Error received from peer  {created_time:"2025-03-14T05:04:16.6051303+00:00", grpc_status:14, grpc_message:"failed to connect to all addresses; last error: UNAVAILABLE: ipv4:127.0.0.1:8080: ConnectEx: Connection refused (No connection could be made because the target machine actively refused it.\r\n -- 10061)"}"
>


	Instead, use the `flower-supernode` CLI command to start a SuperNode as shown below:

		$ flower-supernode --insecure --superlink='<IP>:<PORT>'

	To view all available options, run:

		$ flower-supernode --help

	Using `start_client()` is deprecated.

            This is a deprecated feature. It will be removed
            entirely in future versions of Flower.
        


Client disconnected, retrying... <_MultiThreadedRendezvous of RPC that terminated with:
	status = StatusCode.UNAVAILABLE
	details = "failed to connect to all addresses; last error: UNAVAILABLE: ipv4:127.0.0.1:8080: ConnectEx: Connection refused (No connection could be made because the target machine actively refused it.
 -- 10061)"
	debug_error_string = "UNKNOWN:Error received from peer  {created_time:"2025-03-14T05:04:18.8524578+00:00", grpc_status:14, grpc_message:"failed to connect to all addresses; last error: UNAVAILABLE: ipv4:127.0.0.1:8080: ConnectEx: Connection refused (No connection could be made because the target machine actively refused it.\r\n -- 10061)"}"
>
